# IOT-IPS

The script for iot-ips project

Current use a simple fully-connected classifer with rssi + csi information.

In [2]:
import os
import sys
import csv
import math
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

/home/rtu/anaconda3/envs/habitat/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Data

## Define the input format

In [4]:
def parser(rssi, csi, position):
    '''
    description:
        this is a function to rearrange the raw data into the form of your model input
        so that you can easily design the input format and arrange the input data.
        
    input:
        rssi: the averaged rssi in one collect file, stored in numpy array
        csi: an array consists of all csi data in one collect file, stored in a 2D numpy array
        position: nparray(x,y)
        Note: both the rssi and csi belong to the same location
        
    output:
        x: the list containing tensor inputs of the model
        y: the correspond ground truth tensors
    '''
    
    x = []
    y = []
    
    for csi_slice in csi:
        signal = np.concatenate((rssi,csi_slice))
        pos = np.array(position, dtype=np.int32)
        
        x.append(signal)
        y.append(pos)
    
    # to tensor
    x = [torch.Tensor(i) for i in x]
    y = [torch.LongTensor(i) for i in y]
        
    return x, y # use rssi data only as example

## Dataset

In [5]:
class Dataset(Dataset):
    
    def __init__(self, path, parser):
        filenames = os.listdir(path)
        self.parser = parser
        
        x_ = []
        y_ = []
        
        for name in filenames:
            if 'csi' in name:
                file = os.path.join(path, name)
                x,y = self.readOne(file)
                x_.extend(x)
                y_.extend(y)
        
        self.x = x_
        self.y = y_
                      
    def readOne(self, fname):
        
        if 'csi' in fname:
            csi = fname
            rssi = fname.replace('csi','rssi')
        elif 'rssi' in fname:
            rssi = fname
            csi = fname.replace('rssi','csi')
        
        # parse the position from the file name
        position = csi[csi.find('csi')+3: csi.rfind('.')].split('_')
        position = np.asfarray([float(i) for i in position])
        
        rssi_f = open(rssi)
        csi_f = open(csi)
        rssilines = rssi_f.readlines()
        csilines = csi_f.readlines()
        
        # Read the rssi and average
        row = 0
        rssi_data = np.zeros(8, dtype=float)
        for line in rssilines:
            rssi_raw = line.split(',')
            try:
                rssi_data += np.asfarray([int(i) for i in rssi_raw])
                row += 1
            except:
                continue
        rssi_data /= row
            
        # Read the csi
        csi_data = []
        for line in csilines:
            csi_raw = line.split(',')[-2][1:-2]  # skip the brackets and space
            csi_str = csi_raw.split(' ')
            csi_float = np.asfarray([int(i) for i in csi_str])
            csi_data.append(csi_float)
        csi_data = np.asfarray(csi_data)
        
        rssi_f.close()
        csi_f.close()
        
        x,y = self.parser(rssi_data, csi_data, position)
        
        return x,y
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Model

In [6]:
class Model(nn.Module):
    def __init__(self, input_dim=136, hidden=32):
        super(Model, self).__init__()
        
        layers = [
            nn.Linear(input_dim,hidden),
            nn.LeakyReLU(),
            nn.Linear(hidden,27),
        ]
        
        self.xlayers = nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.xlayers(x)
        return x

# Hyper Parameters

In [10]:
lr = 0.0005
momentum = 0.03
n_epochs = 10000
batch_size = 64
valid_epoch = 100
save_path = './test.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
train_dataset = Dataset('data/train', parser)
test_dataset = Dataset('data/test', parser)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

model = Model().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum) 

best_loss, step, early_stop_count = math.inf, 0, 0

train_pbar = tqdm(range(n_epochs), position=0, leave=True)
for epoch in train_pbar:
    
    model.train()
    
    acc_record = []
    loss_record = []

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x+torch.rand(x.size()).to(device))  
        
        y = (y[:,0]+y[:,1]*3)
        loss = criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        step += 1
        
        _,pred = torch.max(pred, 1)
        acc = (pred==y).sum().item()
        acc_record.append(acc)
        loss_record.append(loss.detach().item())

        train_pbar.set_postfix({'loss': loss.detach().item()})
    train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
    
    mean_train_loss = sum(loss_record)/len(loss_record)
    mean_train_acc = sum(acc_record)/len(train_dataset)

    if epoch % valid_epoch==0 or epoch==n_epochs-1:

        acc_record = []
        
        model.eval() # Set your model to evaluation mode.
        loss_record = []
        with torch.no_grad():
            for x, y in valid_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                
                y = (y[:,0]+y[:,1]*3)
                _,pred = torch.max(pred, 1)
                acc = (pred==y).sum().item()
                acc_record.append(acc)

        mean_valid_acc = sum(acc_record)/len(test_dataset)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Train acc: {mean_train_acc:.4f}, Valid acc: {mean_valid_acc:.4f}')
    
        if mean_train_loss < best_loss:
            best_loss = mean_train_loss
            torch.save(model.state_dict(), save_path)
            print('Saving model with loss {:.3f}...'.format(best_loss))


Epoch [4/10000]:   0%|              | 4/10000 [00:00<09:54, 16.82it/s, loss=3.3]

Epoch [1/10000]: Train loss: 4.5994, Train acc: 0.0334, Valid acc: 0.0333
Saving model with loss 4.599...


Epoch [104/10000]:   1%|         | 104/10000 [00:05<09:20, 17.66it/s, loss=2.69]

Epoch [101/10000]: Train loss: 2.8005, Train acc: 0.2219, Valid acc: 0.0100
Saving model with loss 2.801...


Epoch [204/10000]:   2%|▏        | 204/10000 [00:11<10:11, 16.03it/s, loss=2.26]

...

Epoch [1601/10000]: Train loss: 0.0537, Train acc: 1.0000, Valid acc: 0.0000
Saving model with loss 0.054...


Epoch [1704/10000]:  17%|▊    | 1703/10000 [01:36<08:11, 16.87it/s, loss=0.0472]

Epoch [1701/10000]: Train loss: 0.0471, Train acc: 1.0000, Valid acc: 0.0000
Saving model with loss 0.047...


Epoch [1743/10000]:  17%|▊    | 1742/10000 [01:39<07:50, 17.56it/s, loss=0.0344]


KeyboardInterrupt: 

In [15]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

model.eval() # Set your model to evaluation mode.
loss_record = []
val_acc = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        
        pred = model(x)
        _, pred = torch.max(pred, 1) 
        pred = pred.cpu().detach().numpy()[0]
        
        print(f'Prediction: {np.asarray([pred%3, pred//3])}')
        print(f'GT        : {y.cpu().detach().numpy()[0]}\n\n')

Prediction: [0 8]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [0 8]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [0 8]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: [2 3]
GT        : [1 7]


Prediction: 

GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [1 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [1 7]
GT        : [-1  9]


Prediction: [1 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [1 7]
GT        : [-1  9]


Prediction: [1 7]
GT        : [-1  9]


Prediction: [0 7]
GT        : [-1  9]


Prediction: [1 7]
GT        : [-1  9]


Prediction: [1 7]
